# План работы над проектом
1. [Описание проекта](#1-описание-проекта)
2. [Импорт библиотек](#2импорт-библиотек)
3. [Обзор данных](#3-обзор-данных)
4. [Предобработка и подготовка данных](#4-предобработка-и-подготовка-данных)
5. [Исследовательский анализ данных](#исследовательский-анализ-данных)
6. [A\B тестирование и результаты](#ab-тестирование-и-результаты)
7. [Проверка статистической разницы z-критерием](#проверка-статистической-разницы-долей-z-критерием)
8. [Общий вывод](#общий-вывод)

## 1. Описание проекта

### Оценка результатов A/B-теста

Описание проекта.
В нашем распоряжении датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов.

Цель исследования :
- Оценка корректности проведения теста 
- Анализ результатов теста

Техническое задание:
* Название теста: recommender_system_test;
* группы: А — контрольная, B — новая платёжная воронка;
* дата запуска: 2020-12-07;
* дата остановки набора новых пользователей: 2020-12-21;
* дата остановки: 2021-01-04;
* аудитория: 15% новых пользователей из региона EU;
* назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
* ожидаемое количество участников теста: 6000.
* ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
* конверсии в просмотр карточек товаров — событие product_page,
* просмотры корзины — product_cart,
* покупки — purchase.
* Ход исследования:

Данные находятся в файлах:

* `final_ab_events.csv`
* `final_ab_project_marketing_events.csv`
* `final_ab_new_users.csv`
* `final_ab_participants.csv`

О качестве данных ничего не известно. 

## 2.Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

# Стандартные импорты plotly
import plotly
import plotly.graph_objs as go
import plotly.express as px

from plotly.subplots import make_subplots
from plotly.offline import iplot

# На всякий случай если не смогу в plotly
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns

# Статистика
import math
from scipy import stats as st
from statsmodels.stats.proportion import proportions_ztest

import warnings
warnings.filterwarnings('ignore')

In [2]:
#столбцы и строки полностью, формат округлен
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

pd.set_option('display.max_colwidth', None) #вывод значений без сокращений
pd.set_option('display.max_columns', None)  # Отображение всех столбцов без ограничений по ширине
pd.set_option('display.expand_frame_repr', False)  # Отключение разбиения строк на несколько строк


## 3. Обзор данных

### Загрузка данных

In [3]:
try:
    events = pd.read_csv('/datasets/final_ab_events.csv')  
    marketing_events = pd.read_csv('/datasets/ab_project_marketing_events.csv')  
    new_users = pd.read_csv('/datasets/final_ab_new_users.csv')  
    participants = pd.read_csv('/datasets/final_ab_participants.csv')  
except:
    events = pd.read_csv('https://code.s3.yandex.net//datasets/final_ab_events.csv')
    marketing_events = pd.read_csv('https://code.s3.yandex.net//datasets/ab_project_marketing_events.csv')
    new_users = pd.read_csv('https://code.s3.yandex.net//datasets/final_ab_new_users.csv')
    participants = pd.read_csv('https://code.s3.yandex.net//datasets/final_ab_participants.csv')

### Функция

In [4]:
# Создам функцию для проверки датасета
def display_dataset_info(dataset):
    print(f"Первые несколько строк датасета:\n{dataset.head()}")
    print("\n----------------------------------------")
    print(f"Размер датасета: {dataset.shape}")
    print("\n----------------------------------------")
    print(f"Доля пустых строк:\n{dataset.isna().mean().sort_values(ascending=False)}")
    print("\n----------------------------------------")
    print(f"Типы данных в датасете:\n{dataset.dtypes}")
    print("\n----------------------------------------")
    print(f"Количество дубликатов: {dataset.duplicated().sum()}")
    print("\n----------------------------------------")
    print("Описательная статистика датасета:")
    print(dataset.describe(include='all'))


### Датасет `events`

In [5]:
display_dataset_info(events)

Первые несколько строк датасета:
            user_id             event_dt event_name  details
0  E1BDDCE0DAFA2679  2020-12-07 20:22:03   purchase    99.99
1  7B6452F081F49504  2020-12-07 09:22:53   purchase     9.99
2  9CD9F34546DF254C  2020-12-07 12:59:29   purchase     4.99
3  96F27A054B191457  2020-12-07 04:02:40   purchase     4.99
4  1FD7660FDF94CA1F  2020-12-07 10:15:09   purchase     4.99

----------------------------------------
Размер датасета: (440317, 4)

----------------------------------------
Доля пустых строк:
details      0.86
user_id      0.00
event_dt     0.00
event_name   0.00
dtype: float64

----------------------------------------
Типы данных в датасете:
user_id        object
event_dt       object
event_name     object
details       float64
dtype: object

----------------------------------------
Количество дубликатов: 0

----------------------------------------
Описательная статистика датасета:
                 user_id             event_dt event_name   details
coun

### Вывод по `events`: 
- Датасет 4 столбца 440317 строк
- Из четырех полей `details` имеет 86% пропусков (нужно проработать)
- `event_dt` имеет тип данных *object* нужно изменить на *datetime*
- Явных дубликатов нет
- Имеем 4 уникальных `event_name` и 440317 уникальных юзеров

### Датасет `marketing_events`

In [6]:
display_dataset_info(marketing_events)

Первые несколько строк датасета:
                           name                   regions    start_dt   finish_dt
0      Christmas&New Year Promo             EU, N.America  2020-12-25  2021-01-03
1  St. Valentine's Day Giveaway  EU, CIS, APAC, N.America  2020-02-14  2020-02-16
2        St. Patric's Day Promo             EU, N.America  2020-03-17  2020-03-19
3                  Easter Promo  EU, CIS, APAC, N.America  2020-04-12  2020-04-19
4             4th of July Promo                 N.America  2020-07-04  2020-07-11

----------------------------------------
Размер датасета: (14, 4)

----------------------------------------
Доля пустых строк:
name        0.00
regions     0.00
start_dt    0.00
finish_dt   0.00
dtype: float64

----------------------------------------
Типы данных в датасете:
name         object
regions      object
start_dt     object
finish_dt    object
dtype: object

----------------------------------------
Количество дубликатов: 0

------------------------------------

### Вывод по `marketing_events`:
- Датасет 4 столбца 14 строк
- Пустых строк нет
- Типы данных `start_dt` и `finish_dt` *object* а там даты
- Явных дубликатов нет
- Содержит 14 маркетинговых компаний в шести регионах и 14 дат(начала и конца события).

### Датасет `new_user`

In [7]:
display_dataset_info(new_users)

Первые несколько строк датасета:
            user_id  first_date     region   device
0  D72A72121175D8BE  2020-12-07         EU       PC
1  F1C668619DFE6E65  2020-12-07  N.America  Android
2  2E1BF1D4C37EA01F  2020-12-07         EU       PC
3  50734A22C0C63768  2020-12-07         EU   iPhone
4  E1BDDCE0DAFA2679  2020-12-07  N.America   iPhone

----------------------------------------
Размер датасета: (61733, 4)

----------------------------------------
Доля пустых строк:
user_id      0.00
first_date   0.00
region       0.00
device       0.00
dtype: float64

----------------------------------------
Типы данных в датасете:
user_id       object
first_date    object
region        object
device        object
dtype: object

----------------------------------------
Количество дубликатов: 0

----------------------------------------
Описательная статистика датасета:
                 user_id  first_date region   device
count              61733       61733  61733    61733
unique             61733

### Вывод по `new_users`:
- Датасет 4 столбца 61733 строки
- Пустых строк нет
- Тип данных `first_date` *object* а там дата
- Явных дубликатов нет
- Содержит 61733 юзера по 4 регионам, которые используют 4 вида девайсов и приходили 17 дней.

### Датасет `participants`

In [8]:
display_dataset_info(participants)

Первые несколько строк датасета:
            user_id group                  ab_test
0  D1ABA3E2887B6A73     A  recommender_system_test
1  A7A3664BD6242119     A  recommender_system_test
2  DABC14FDDFADD29E     A  recommender_system_test
3  04988C5DF189632E     A  recommender_system_test
4  482F14783456D21B     B  recommender_system_test

----------------------------------------
Размер датасета: (18268, 3)

----------------------------------------
Доля пустых строк:
user_id   0.00
group     0.00
ab_test   0.00
dtype: float64

----------------------------------------
Типы данных в датасете:
user_id    object
group      object
ab_test    object
dtype: object

----------------------------------------
Количество дубликатов: 0

----------------------------------------
Описательная статистика датасета:


                 user_id  group            ab_test
count              18268  18268              18268
unique             16666      2                  2
top     0FDFDA0B2DEC2D91      A  interface_eu_test
freq                   2   9655              11567


In [9]:
participants.groupby('user_id').agg({'ab_test': ['nunique', 'unique']}).head() 

ab_test                                              
                 nunique                                        unique
user_id                                                               
0002CE61FF2C4011       1                           [interface_eu_test]
000ABE35EE11412F       1                     [recommender_system_test]
001064FEAAB631A1       2  [recommender_system_test, interface_eu_test]
0010A1C096941592       1                     [recommender_system_test]
001C05E87D336C59       1                     [recommender_system_test]

### Вывод по `participants`:
- Датасет 3 стоблца 18268 строк
- Пропусков нет
- Тип данных в норме
- Явных дубликатов нет
- Содержит две группы тестирования и два теста
- Количество юзеров 18268, а уникальных юзеров 16666(нужно посмотреть) 1602 попали или в два теста, или в две группы.

## 4. Предобработка и подготовка  данных 

### Датасет `events`

In [10]:
events.groupby('event_name').count() 

,user_id,event_dt,details
event_name,,,
login,189552,189552,0
product_cart,62462,62462,0
product_page,125563,125563,0
purchase,62740,62740,62740


Поле `details` заполнено только тогда когда прошло событие `purchase`(покупка) и заносится сумма
- пропуски оставляю как есть

In [11]:
events['event_dt']= pd.to_datetime(events['event_dt']) # Меняем тип данных столбца 
events.dtypes # Прверим изменения

user_id               object
event_dt      datetime64[ns]
event_name            object
details              float64
dtype: object

In [12]:
print(f"Начальная дата  наблюдений за событими: {events['event_dt'].min().date()}")
print(f"Конечная дата наблюдений за событиями : {events['event_dt'].max().date()}")

Начальная дата  наблюдений за событими: 2020-12-07
Конечная дата наблюдений за событиями : 2020-12-30


In [13]:
events['details'].value_counts() # значения покупок

4.99      46362
9.99       9530
99.99      5631
499.99     1217
Name: details, dtype: int64

4 вида покупок(цены) 4.99 - 499.99

In [14]:
events.sample()

,user_id,event_dt,event_name,details
306175,77CDE5B7C785FC62,2020-12-14 11:00:17,login,NaN


- Датасет 4 столбца 440317 строк
- Пропуски в данных не являются ошибкой в процессе загрузки данных и не требуют заполнения. Они возникают, когда отсутствуют детали о действиях пользователя, кроме покупки. В случае покупки, информация о стоимости товара сохраняется, но для других действий информация может отсутствовать.
- Имеем 4 уникальных `event_name` 
1. login
2. product_card (карточка продукта)
3. product_page (страница продукта)
4. purshace     (покупка)
- 4 вида покупок(цены) 4.99, 9.99, 99.99, 499.99

По ТЗ нам нужно остановить тест 4 января 2021, но данные у нас только до 30 декабря 2020, возникает  отклонение от ТЗ и нужно это учитывать при дальнейшем анализе .

### Датасет `marketing_events`

In [15]:
marketing_events[['start_dt', 'finish_dt']] = marketing_events[['start_dt', 'finish_dt']].apply(pd.to_datetime, errors='coerce') # Меняем тип данных столбца 
marketing_events.dtypes # Проверяем

name                 object
regions              object
start_dt     datetime64[ns]
finish_dt    datetime64[ns]
dtype: object

In [16]:
marketing_events # Посмотрим на вский случай

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


In [17]:
print(f"Минимальная дата старта компании: {marketing_events['start_dt'].min().date()}")
print(f"Максимальная дата старта компании: {marketing_events['start_dt'].max().date()}")
print(f"Минимальная дата окончания компании: {marketing_events['finish_dt'].min().date()}")
print(f"Максимальная дата окончания компании: {marketing_events['finish_dt'].max().date()}")

Минимальная дата старта компании: 2020-01-25
Максимальная дата старта компании: 2020-12-30
Минимальная дата окончания компании: 2020-02-07
Максимальная дата окончания компании: 2021-01-07


In [18]:
marketing_events['regions'].value_counts()

APAC                        4
EU, CIS, APAC, N.America    3
EU, N.America               2
EU, CIS, APAC               2
CIS                         2
N.America                   1
Name: regions, dtype: int64

In [19]:
marketing_events['regions'].unique()

array(['EU, N.America', 'EU, CIS, APAC, N.America', 'N.America', 'APAC',
       'EU, CIS, APAC', 'CIS'], dtype=object)

14 названий событий, всего мы видим 4 региона, но так как некоторые события проводились сразу в нескольких регионах видим шесть уникальных сочетаний

Маркетинговые события проходят в различных регионах, таких как Европа, Северная Америка, СНГ и Азиатские страны. Эти события проходили в период с 25 января по 30 декабря 2020 года. Окончание кампаний пришлось на период с 7 февраля 2020 года по 7 января 2021 года.

С временем проведения нашего теста имеет пересечение только событие "Christmas&New Year Promo" которое проходило для двух регионов EU и N.America с 25 декабря по 3 января. И хотя для региона CIS с 30 декабря по 7 января проводилось маркетинговое событие "CIS New Year Gift Lottery" оно не попадает в пересечение так как у нас данные по событиям как мы выяснили только до 30 декабря и отклоняются от ТЗ.


### Датасет `new_user`

In [20]:
new_users['first_date'] = pd.to_datetime(new_users['first_date']) # Меняем тип столбца
new_users.dtypes

user_id               object
first_date    datetime64[ns]
region                object
device                object
dtype: object

In [21]:
new_users.sample()

,user_id,first_date,region,device
17328,F7E3D72B13C28F62,2020-12-21,EU,Mac


In [22]:
print(f"Минимальная прихода нового пользователя: {new_users['first_date'].min().date()}")
print(f"Максимальная прихода нового пользователя: {new_users['first_date'].max().date()}")

Минимальная прихода нового пользователя: 2020-12-07
Максимальная прихода нового пользователя: 2020-12-23


In [23]:
new_users['region'].value_counts()

EU           46270
N.America     9155
CIS           3155
APAC          3153
Name: region, dtype: int64

In [24]:
new_users['device'].value_counts()

Android    27520
PC         15599
iPhone     12530
Mac         6084
Name: device, dtype: int64

In [25]:
new_users.shape

(61733, 4)

56470 юзера по 4 регионам, которые используют 4 вида девайсов и приходили 17 дней.
- Новые пользователи приходили с 7 по 23 декабря 2020 года.
- С четырех регионов львиная доля с Европы
- С четырех девайсов под управлением разных операционных систем
- Настольные поделены на Mac и PC

### Датасет `participants`

In [26]:
participants.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


In [27]:
participants.groupby('user_id').agg({'group' : 'nunique'}).query('group>1').count() # Количество пользователей попавших в обе группы тестирования

group    776
dtype: int64

In [28]:
participants.groupby('user_id').agg({'ab_test' : 'nunique'}).query('ab_test>1').count() # Количество пользователей попавших в оба теста

ab_test    1602
dtype: int64

In [29]:
participants['group'].value_counts() # Посмотрим сколько пользователей по группам

A    9655
B    8613
Name: group, dtype: int64

In [30]:
participants['group'].value_counts(normalize=True) * 100 # Как распределились доли

A   52.85
B   47.15
Name: group, dtype: float64

## Вывод:
Была произведена замена типа данных в столбцах с датой, преобразовав их в формат даты и времени. В данных отсутствуют дубликаты, а пропуски не являются ошибкой и не требуют заполнения. Они возникают, когда отсутствуют детали о действиях пользователя, кроме покупки. 
С временем проведения нашего теста имеет пересечение только событие "Christmas&New Year Promo" которое проходило для двух регионов EU и N.America с 25 декабря по 3 января. И хотя для региона CIS с 30 декабря по 7 января проводилось маркетинговое событие "CIS New Year Gift Lottery" оно не попадает в пересечение так как у нас данные по событиям как мы выяснили только до 30 декабря и отклоняются от ТЗ.
Есть пользователи которые попали как и в оба теста "recommender_system_test, interface_eu_test" 1602, так и в обе группы тестирования 776.

## Проверка пользователей которые будут участвовать в тесте

Создам общую таблицу для дальнейшего анализа.
используя тип объединения 'left', чтобы сохранить все строки из participants и добавить соответствующие значения из других таблиц, если они доступны.

In [31]:
ab_test = (participants
               .merge(events, on='user_id', how='left')
               .merge(new_users, on='user_id', how='left')
               .reset_index(drop=True)
          )
ab_test.head()

,user_id,group,ab_test,event_dt,event_name,details,first_date,region,device
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,purchase,99.99,2020-12-07,EU,PC
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-25 00:04:56,purchase,4.99,2020-12-07,EU,PC
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:29,product_cart,NaN,2020-12-07,EU,PC
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-25 00:04:57,product_cart,NaN,2020-12-07,EU,PC
4,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,product_page,NaN,2020-12-07,EU,PC


In [32]:
ab_test.shape

(110368, 9)

In [33]:
ab_test.duplicated().sum() # Проверим на дубликаты

0

Так как нам нужны участники которые прошли `recommender_system_test` по ТЗ

In [34]:
ab_test_final = ab_test[ab_test['ab_test']=='recommender_system_test']

In [35]:
ab_test_final['ab_test'].value_counts()

recommender_system_test    27724
Name: ab_test, dtype: int64

In [36]:
ab_test_final.shape

(27724, 9)

По ТЗ известно, что дата запуска теста: 
- 7 декабря 2020 года, а дата остановки набора новых пользователей: 2020-12-21. 
- Соответственно, нас интересуют действия пользователей только в рамках указанного периода. 
- Проверим таблицу `ab_test_final` на даты и если будет нужно отфильтруем согластно ТЗ.

In [37]:
print(ab_test_final['first_date'].min())
print(ab_test_final['first_date'].max())

2020-12-07 00:00:00
2020-12-21 00:00:00


Так как даты согласуются с ТЗ дополнительная фильтрация не нужна.

In [38]:
ab_test_final.shape

(27724, 9)

Проведем проверку и оставим только те события, у которых с момента регистраци прошло 14 дней

In [39]:
# Фильтрация данных для заданного периода времени и всех пользователей
ab_test_final = ab_test_final[(ab_test_final['event_dt'] >= ab_test_final['first_date'])  & 
                        (ab_test_final['event_dt'] <= ab_test_final['first_date'] + pd.DateOffset(days=14))]

ab_test_final.shape

(23856, 9)

Также нам необходимо посмотреть информацию о том, какие маркетинговые кампании проводились в рассматриваемый период (и проводились ли вообще)
с 7 декабря по 30 декабря 2020

In [40]:
min_dt = pd.to_datetime('2020-12-07', format='%Y-%m-%d')
max_dt = pd.to_datetime('2021-12-30', format='%Y-%m-%d')

# Фильтрация маркетинговых событий
filtered_events = marketing_events[
    (marketing_events['regions'].str.contains("EU")) &
    ((marketing_events['start_dt'] >= min_dt) | (marketing_events['finish_dt'] >= min_dt)) &
    ((marketing_events['start_dt'] <= max_dt) | (marketing_events['finish_dt'] <= max_dt))
]

# Добавление информации о маркетинговых событиях в ab_test_final
ab_test_final['marketing_event'] = 'нет событий'
for i, row in ab_test_final.iterrows():
    event_date = row['event_dt']
    region = row['region']
    
    for j, event_row in filtered_events.iterrows():
        start_date = event_row['start_dt']
        end_date = event_row['finish_dt']
        regions = event_row['regions'].split(', ')
        
        if start_date <= event_date <= end_date and region in regions:
            ab_test_final.at[i, 'marketing_event'] = event_row['name']
            break


In [41]:
ab_test_final.head(7) # Для сравнения 

,user_id,group,ab_test,event_dt,event_name,details,first_date,region,device,marketing_event
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,purchase,99.99,2020-12-07,EU,PC,нет событий
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:29,product_cart,NaN,2020-12-07,EU,PC,нет событий
4,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,product_page,NaN,2020-12-07,EU,PC,нет событий
6,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,login,NaN,2020-12-07,EU,PC,нет событий
8,A7A3664BD6242119,A,recommender_system_test,2020-12-20 15:46:06,product_page,NaN,2020-12-20,EU,iPhone,нет событий
9,A7A3664BD6242119,A,recommender_system_test,2020-12-21 00:40:59,product_page,NaN,2020-12-20,EU,iPhone,нет событий
10,A7A3664BD6242119,A,recommender_system_test,2020-12-25 05:19:45,product_page,NaN,2020-12-20,EU,iPhone,Christmas&New Year Promo


По результатам анализа, не рекомендуется удалять такие события. Новые пользователи были привлечены в наш тест задолго до начала маркетинговых событий, а основные события обычно происходят в первые несколько дней использования. Таким образом, влияние этих событий на наш тест является минимальным.

При обзоре данных мы выявили что пользователь может участвовать в двух тестах, но у нас нет данных о его дате проведения. Возможно, некоторые пользователи попали и в этот тест, и в наш тест, и если даты пересеклись, это могло повлиять на результаты. Поэтому мы должны оставить только пользователей, участвовавших в одном A/B-тестировании.

In [42]:
ab_test_final['ab_test'].value_counts()


recommender_system_test    23856
Name: ab_test, dtype: int64

In [43]:
ab_test_final['ab_test'].unique()

array(['recommender_system_test'], dtype=object)

In [44]:
# Найти пользователей с более чем одним уникальным тестом
multi_test_users = participants.groupby('user_id')['ab_test'].nunique()
multi_test_users = multi_test_users[multi_test_users > 1].index

# Исключить пользователей, у которых более одного уникального теста, из ab_test_final
ab_test_final = ab_test_final[~ab_test_final['user_id'].isin(multi_test_users)].reset_index(drop=True)

In [45]:
ab_test_final.shape

(18154, 10)

In [46]:
ab_test_final.groupby('user_id').agg({'group' : 'nunique'}).query('group>1').count() # Количество пользователей попавших в обе группы тестирования

group    0
dtype: int64

In [47]:
len(ab_test_final) / len(ab_test) * 100

16.448608292258626

In [48]:
ab_test['user_id'].nunique() - ab_test_final['user_id'].nunique()

13878

In [49]:
# Подсчет количества уникальных пользователей по группам в исходном датасете
users_ab_test = ab_test.groupby('group')['user_id'].nunique()

# Подсчет количества уникальных пользователей по группам в окончательном датасете
users_ab_test_final = ab_test_final.groupby('group')['user_id'].nunique()

# Создание графика
fig = go.Figure()

fig.add_trace(go.Bar(
    x=['A', 'B'],
    y=[users_ab_test['A'], users_ab_test['B']],
    name='ab_test',
    text=[users_ab_test['A'], users_ab_test['B']],
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=['A', 'B'],
    y=[users_ab_test_final['A'], users_ab_test_final['B']],
    name='ab_test_final',
    text=[users_ab_test_final['A'], users_ab_test_final['B']],
    textposition='auto'
    
))

fig.update_layout(
    title='Количество уникальных пользователей по группам',
    xaxis_title='Группа',
    yaxis_title='Количество уникальных пользователей',
    barmode='group',
    width=1200, height=700
)

fig.show()



Пришлось удалить примерно 35% данных, так как около 35% пользователей участвовали в двух A/B-тестах. Это позволило очистить отфильтрованные данные от временно пересекающихся событий, которые могли повлиять на результаты, примерно на 35%. Пересечений внутри нашего теста нет.При очищении данных потеряли 13878 пользователй.

In [50]:
ab_test_final['region'].value_counts() # Посмотрим на совершенные события по регионам

EU           16918
N.America      815
APAC           225
CIS            196
Name: region, dtype: int64

In [51]:
user_eu_test = ab_test_final[ab_test_final['region'] == 'EU']['user_id'].nunique()
user_eu_new = new_users[new_users['region'] == 'EU']['user_id'].nunique()
percentage = user_eu_test / user_eu_new * 100
print(f"Доля участников нашего теста среди новых пользователей из региона EU: {percentage:.2f}%")

Доля участников нашего теста среди новых пользователей из региона EU: 5.61%


## Вывод:
- В процессе анализа данных обнаруженно, что некоторые пользователи участвовали в A/B-тестировании одновременно с проведением маркетингового события "Christmas&New Year Promo". Учитывая, что новые пользователи были привлечены в наш тест задолго до начала маркетинговых событий, а основные события обычно происходят в первые несколько дней использования. Влияние этих событий является минимальным, данные оставили.

- Также стало ясно, что некоторые пользователи участвовали в двух A/B-тестах, но у нас отсутствуют данные о дате проведения второго теста. Если даты тестов пересекаются, это может оказать влияние на результаты. Поэтому решил оставить только тех пользователей, которые участвовали в одном A/B-тестировании.

- В результате удаления пользователей, участвовавших в маркетинговом событии и/или двух A/B-тестах, мы удалили около 35% данных. Это позволило нам очистить отфильтрованные данные от временно пересекающихся событий, которые могли повлиять на результаты.

- Доля участников нашего теста среди новых пользователей из региона EU: 5.61%. Что тоже не совпадает с нашим ТЗ таких пользователей должно быть 15 %.

Таким образом, мы получили очищенные данные, на основе которых можно проводить более точный анализ и делать выводы о результативных метриках нашего A/B-тестирования.

## Распределение пользователей по группам

In [52]:
group_a = ab_test_final.query('group == "A"')
group_a_unique_users = group_a['user_id'].nunique()
group_a_percentage = group_a_unique_users / len(ab_test_final['user_id'].unique()) * 100

group_a_eu = group_a.query('region == "EU"')
group_a_eu_unique_users = group_a_eu['user_id'].nunique()
group_a_eu_percentage = group_a_eu_unique_users / len(ab_test_final.query('region == "EU"')['user_id'].unique()) * 100

group_a_na = group_a.query('region == "N.America"')
group_a_na_unique_users = group_a_na['user_id'].nunique()
group_a_na_percentage = group_a_na_unique_users / len(ab_test_final.query('region == "N.America"')['user_id'].unique()) * 100

group_a_ap = group_a.query('region == "APAC"')
group_a_ap_unique_users = group_a_ap['user_id'].nunique()
group_a_ap_percentage = group_a_ap_unique_users / len(ab_test_final.query('region == "APAC"')['user_id'].unique()) * 100

group_a_cis = group_a.query('region == "CIS"')
group_a_cis_unique_users = group_a_cis['user_id'].nunique()
group_a_cis_percentage = group_a_cis_unique_users / len(ab_test_final.query('region == "CIS"')['user_id'].unique()) * 100




group_b = ab_test_final.query('group == "B"')
group_b_unique_users = group_b['user_id'].nunique()
group_b_percentage = group_b_unique_users / len(ab_test_final['user_id'].unique()) * 100

group_b_eu = group_b.query('region == "EU"')
group_b_eu_unique_users = group_b_eu['user_id'].nunique()
group_b_eu_percentage = group_b_eu_unique_users / len(ab_test_final.query('region == "EU"')['user_id'].unique()) * 100

group_b_na = group_b.query('region == "N.America"')
group_b_na_unique_users = group_b_na['user_id'].nunique()
group_b_na_percentage = group_b_na_unique_users / len(ab_test_final.query('region == "N.America"')['user_id'].unique()) * 100

group_b_ap = group_b.query('region == "APAC"')
group_b_ap_unique_users = group_b_ap['user_id'].nunique()
group_b_ap_percentage = group_b_ap_unique_users / len(ab_test_final.query('region == "APAC"')['user_id'].unique()) * 100

group_b_cis = group_b.query('region == "CIS"')
group_b_cis_unique_users = group_b_cis['user_id'].nunique()
group_b_cis_percentage = group_b_cis_unique_users / len(ab_test_final.query('region == "CIS"')['user_id'].unique()) * 100


print(f"Количество уникальных пользователей из группы A: {group_a_unique_users}")
print(f"Доля пользователей из группы A: {group_a_percentage:.2f}%")
print(f"Количество уникальных пользователей из группы B: {group_b_unique_users}")
print(f"Доля пользователей из группы B: {group_b_percentage:.2f}%")
print("----------------------------------------")

print(f"Количество уникальных пользователей из группы A в регионе EU: {group_a_eu_unique_users}")
print(f"Доля пользователей из группы A в регионе EU: {group_a_eu_percentage:.2f}%")
print(f"Количество уникальных пользователей из группы B в регионе EU: {group_b_eu_unique_users}")
print(f"Доля пользователей из группы B в регионе EU: {group_b_eu_percentage:.2f}%")
print("----------------------------------------")

print(f"Количество уникальных пользователей из группы A в регионе N.America: {group_a_na_unique_users}")
print(f"Доля пользователей из группы A в регионе N.America: {group_a_na_percentage:.2f}%")
print(f"Количество уникальных пользователей из группы B в регионе N.America: {group_b_na_unique_users}")
print(f"Доля пользователей из группы B в регионе N.America: {group_b_na_percentage:.2f}%")
print("----------------------------------------")

print(f"Количество уникальных пользователей из группы A в регионе APAC: {group_a_ap_unique_users}")
print(f"Доля пользователей из группы A в регионе APAC: {group_a_ap_percentage:.2f}%")
print(f"Количество уникальных пользователей из группы B в регионе APAC: {group_b_ap_unique_users}")
print(f"Доля пользователей из группы B в регионе APAC: {group_b_ap_percentage:.2f}%")
print("----------------------------------------")

print(f"Количество уникальных пользователей из группы A в регионе CIS: {group_a_cis_unique_users}")
print(f"Доля пользователей из группы A в регионе APAC: {group_a_cis_percentage:.2f}%")
print(f"Количество уникальных пользователей из группы B в регионе CIS: {group_b_cis_unique_users}")
print(f"Доля пользователей из группы B в регионе APAC: {group_b_cis_percentage:.2f}%")



Количество уникальных пользователей из группы A: 2082
Доля пользователей из группы A: 74.68%
Количество уникальных пользователей из группы B: 706
Доля пользователей из группы B: 25.32%
----------------------------------------
Количество уникальных пользователей из группы A в регионе EU: 1939
Доля пользователей из группы A в регионе EU: 74.75%
Количество уникальных пользователей из группы B в регионе EU: 655
Доля пользователей из группы B в регионе EU: 25.25%
----------------------------------------
Количество уникальных пользователей из группы A в регионе N.America: 96
Доля пользователей из группы A в регионе N.America: 80.67%
Количество уникальных пользователей из группы B в регионе N.America: 23
Доля пользователей из группы B в регионе N.America: 19.33%
----------------------------------------
Количество уникальных пользователей из группы A в регионе APAC: 28
Доля пользователей из группы A в регионе APAC: 62.22%
Количество уникальных пользователей из группы B в регионе APAC: 17
Доля 

###  Вывод 
Как видно среди участников теста есть пользователи из других регионов.наличие участников теста из других регионов, кроме EU является отклонением от Т.З

In [53]:
# Группировка данных по столбцам "group" и "region" и подсчет уникальных пользователей
grouped_data = ab_test_final.groupby(['group', 'region']).agg({'user_id': 'nunique'}).reset_index()


# Построение столбчатой диаграммы (бар-графика)
fig = px.bar(grouped_data.sort_values(by='user_id', ascending=False), x='group', y='user_id', color='region', 
             labels={'group': 'Группа', 'user_id': 'Количество уникальных пользователей', 'region': 'Регион'},
             barmode='group', text='user_id',width=1200, height=700)
fig.update_layout(title='Количество уникальных пользователей в каждой группе по регионам')
fig.show()

## Вывод:
В группах A и B присутствуют разные пользователи. Группа A состоит из 2082 человек (примерно 75% от общего числа пользователей), в то время как группа B включает 706 человек (примерно 25% от общего числа пользователей), а также пользователи из региона EU в группе А 1939 человек (примерно 75% от общего числа пользователей) и в группе В 706 человек (примерно 25% от общего числа пользователей). Хотя идеальным было бы равное распределение пользователей между группами в соотношении 1:1 (50% к 50%), текущие размеры групп достаточны для проведения будующих статистических тестов и анализа результатов.

- Здесь также как в пунктах выше, не выполняется требование по количеству человек участвующих в тесте после очистки от ненужных данных их всего 2788 но даже до очистки данных их было не могим больше 6000 человек. Так что мы убрали тех кто мог нам повлиять на результаты и работаем дальше с тем что есть.
- Для дальнейшего анализа, давайте изучим распределение внутри групп A и B по другим факторам. Это позволит нам лучше понять характеристики и различия между этими группами.

## Исследовательский анализ данных

### Распределение количества событий на пользователя

In [54]:
# Выборки групп A и B
group_A_events = ab_test_final.loc[ab_test_final['group'] == 'A', 'event_name']
group_B_events = ab_test_final.loc[ab_test_final['group'] == 'B', 'event_name']

In [55]:
# Получение количества событий на пользователя для каждой группы
group_A_counts = ab_test_final.loc[ab_test_final['group'] == 'A'].groupby('user_id').size()
group_B_counts = ab_test_final.loc[ab_test_final['group'] == 'B'].groupby('user_id').size()

# Создание списка данных для каждой группы
data = [group_A_counts, group_B_counts]

# Создание списка названий групп
groups = ['Group A', 'Group B']

# Построение бокс-плотов
fig = go.Figure()
for i in range(len(data)):
    fig.add_trace(go.Box(y=data[i], name=groups[i]))

fig.update_layout(
    title='Распределение количества событий на пользователя по группам',
    xaxis_title='Группа',
    yaxis_title='Количество событий на пользователя',
    width=900,
    height=600
)

fig.show()

График показывает что распределение событий на пользователя между группами имеет различия.
Пользователи группы А соверершают больше событий чем пользователи группы В на заданном промежудке времени проведения теста.

Проведу дополнительные статистические тесты.

Сформулирую нулевую и альтернативную гипотезы:


- Нулевая гипотеза (H0): Распределение количества событий на пользователя одинаково в выборках A и B.
- Альтернативная гипотеза (H1): Распределение количества событий на пользователя различается в выборках A и B.

- Критерий Колмогорова-Смирнова является непараметрическим статистическим тестом, который позволяет проверить различия между двумя выборками независимо от их распределения.
- Выбор этого инструмента обусловлен его способностью обнаружить статистически значимые различия в распределении, если они существуют.

In [56]:
# Применение критерия Колмогорова-Смирнова
statistic, p_value = st.ks_2samp(group_A_events, group_B_events)

if p_value > 0.05:
    print("Распределение количества событий на пользователя одинаково распределено в выборках A и B.")
else:
    print("Обнаружены статистически значимые различия в распределении количества событий на пользователя между выборками A и B.")

Обнаружены статистически значимые различия в распределении количества событий на пользователя между выборками A и B.


### Вывод:
Согластно проведеному анализу распределение событий между пользователями группы А и группы В имеют статистически значимые различия.

### Проверим число событий в выборках по дням

In [57]:
# Группировка событий по дням и группам
events_per_day_group = ab_test_final.groupby(['event_dt', 'group']).size().reset_index(name='event_count')

# Построение графика временного ряда
fig = px.histogram(events_per_day_group, x='event_dt', y='event_count', color='group', barmode='overlay')

# Настройка внешнего вида графика
fig.update_layout(
    title='Распределение числа событий по дням и группам',
    xaxis_title='Дата',
    yaxis_title='Количество событий',
    
)

# Отображение графика
fig.show()

### Вывод:
Распределение событий между выборками с 07.12.2020 по 14.12.2020 примерно одинаковое по количеству.
- сначала больше у группы В потом у группы А
С 14 декабря по 25 декабря
- события группы В имеют нормальное распределение
- события группы А кратно возрастают их пик приходится 21, 22 декабря и спад на 25 декабря(но событий сущетвенно больше чем группе В)
- После 25 декабря событий практически нет ни по одной из выборок.

### Распределение пользователей по группам и датам событий

In [58]:
fig = make_subplots(rows=2, cols=2, subplot_titles=[
    'login', 'product_page', 'product_cart', 'purchase'
])

fig.update_layout(title='Распределение количества пользователей по группам и датам событий', height=800, title_x=0.50)

for idx, event_name in enumerate(['login', 'product_page', 'product_cart', 'purchase'], 1):
    row = (idx - 1) // 2 + 1
    col = idx % 2 if idx % 2 else 2
    for group_name, group_events in ab_test_final.groupby('group'):
        data = group_events.loc[group_events['event_name'] == event_name, 'event_dt']
        fig.add_trace(go.Histogram(x=data, name=f'Group {group_name}'), row=row, col=col)

fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=1, col=1)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=1, col=2)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=2, col=1)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=2, col=2)
fig.update_yaxes(title_text='Количество пользователей', row=1, col=1)
fig.update_yaxes(title_text='Количество пользователей', row=1, col=2)
fig.update_yaxes(title_text='Количество пользователей', row=2, col=1)
fig.update_yaxes(title_text='Количество пользователей', row=2, col=2)

fig.update_layout(width=1600, height=800)

fig.show()

### Вывод:
Повторяет вывод по числу событий по дням. Дополняя его тем что количество купивших больше чем число посетивших продуктовую карточку.
- Можно сделать предположение, что некоторые пользователи совершают покупку напрямую со страницы продукта, минуя посещение карточки продукта.

### Распределение  пользователей по регионам и датам событий

In [59]:
fig = make_subplots(rows=2, cols=2, subplot_titles=[
    'EU', 'N.America', 'APAC', 'CIS'
])

fig.update_layout(title='Распределение количества пользователей по регионам и датам событий', height=800, title_x=0.50)

for idx, region in enumerate(['EU', 'N.America', 'APAC', 'CIS'], 1):
    row = (idx - 1) // 2 + 1
    col = idx % 2 if idx % 2 else 2
    for group_name, group_events in ab_test_final.groupby('group'):
        data = group_events.loc[group_events['region'] == region, 'event_dt']
        fig.add_trace(go.Histogram(x=data, name=f'Group {group_name}'), row=row, col=col)

fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=1, col=1)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=1, col=2)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=2, col=1)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=2, col=2)
fig.update_yaxes(title_text='Количество пользователей', row=1, col=1)
fig.update_yaxes(title_text='Количество пользователей', row=1, col=2)
fig.update_yaxes(title_text='Количество пользователей', row=2, col=1)
fig.update_yaxes(title_text='Количество пользователей', row=2, col=2)

fig.update_layout(width=1600, height=800)

fig.show()

### Вывод:
Повторяет вывод по числу событий по дням. Дополняя его тем что доля пользователей с EU, больше всех остальных пользователей вместе взятых.
- Пользовтели EU и N.America(две самые большие группы пользователей по количеству), завершают свою активность 25 декабря.
- После 25 декабря активны только пользователи APAC и CIS(но их доля в количестве невелика)

### Распределение пользователей по девайсам и датам событий

In [60]:
fig = make_subplots(rows=2, cols=2, subplot_titles=[
    'PC', 'iPhone', 'Mac', 'Android'
])

fig.update_layout(title='Распределение количества пользователей по девайсам и датам событий', height=800, title_x=0.50)

for idx, device in enumerate(['PC', 'iPhone', 'Mac', 'Android'], 1):
    row = (idx - 1) // 2 + 1
    col = idx % 2 if idx % 2 else 2
    for group_name, group_events in ab_test_final.groupby('group'):
        data = group_events.loc[group_events['device'] == device, 'event_dt']
        fig.add_trace(go.Histogram(x=data, name=f'Group {group_name}'), row=row, col=col)

fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=1, col=1)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=1, col=2)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=2, col=1)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=2, col=2)
fig.update_yaxes(title_text='Количество пользователей', row=1, col=1)
fig.update_yaxes(title_text='Количество пользователей', row=1, col=2)
fig.update_yaxes(title_text='Количество пользователей', row=2, col=1)
fig.update_yaxes(title_text='Количество пользователей', row=2, col=2)

fig.update_layout(width=1600, height=800)

fig.show()

### Вывод:
Повторяет вывод по числу событий по дням.
Дополнение только в количестве пользователей по девайсам
- Android
- PC и Iphone практически в равных долях
- Mac

### Распределение по датам стоимости покупок

In [61]:
fig = make_subplots(rows=2, cols=2, subplot_titles=[
    4.99, 9.99, 99.99, 499.99
])

fig.update_layout(title='Распределение количества пользователей по покупкам/ценам и датам событий', height=800, title_x=0.50)

for idx, details in enumerate([4.99, 9.99, 99.99, 499.99], 1):
    row = (idx - 1) // 2 + 1
    col = idx % 2 if idx % 2 else 2
    for group_name, group_events in ab_test_final.groupby('group'):
        data = group_events.loc[group_events['details'] == details, 'event_dt']
        fig.add_trace(go.Histogram(x=data, name=f'Group {group_name}'), row=row, col=col)

fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=1, col=1)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=1, col=2)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=2, col=1)
fig.update_xaxes(title_text='Дата', tickformat='%d-%m-%y', row=2, col=2)
fig.update_yaxes(title_text='Количество пользователей', row=1, col=1)
fig.update_yaxes(title_text='Количество пользователей', row=1, col=2)
fig.update_yaxes(title_text='Количество пользователей', row=2, col=1)
fig.update_yaxes(title_text='Количество пользователей', row=2, col=2)

fig.update_layout(width=1600, height=800)

fig.show()

### Вывод:
Повторяет вывод по числу событий по дням. Дополняя его тем что после 25 декабря совершалась немного покупок за 4.99
- По всем остальным ценновым категориям последняя дата покупки 25 декабря.
- Доля покупок в категории 499.99 группы В непропорцианально мала.

### Воронка событий

In [62]:
def event_group_pivot(group):
    result = (ab_test_final
              .query('group == @group')
              .groupby('event_name')
              .agg({'user_id': 'nunique'})
              .reset_index()
              .sort_values(by='user_id', ascending=False)
              .reset_index(drop=True)
             )
    result = result.reindex([0, 1, 3, 2])  # Исправление последовательности event_name
    return result

def event_group_ratio(df):
    df['user_id'] = pd.to_numeric(df['user_id'])  # Преобразование столбца user_id в числовой формат
    max_value = df['user_id'].max()  # Максимальное значение в столбце user_id
    df['ratio'] = (df['user_id'] / len(ab_test_final['user_id'].unique()))
    df['conv'] = df['user_id'] / max_value
    df['conv_n_n-1'] = df['user_id'] / df['user_id'].shift(1)  # Конверсия n/n-1
    df['conv_n_n-1'].iloc[0] = 1.00  # Установка первого значения в 1.00
# Таблица для группы A
event_A_pivot = event_group_pivot('A')
event_group_ratio(event_A_pivot)
event_A_pivot.columns = ['event_name', 'group a_количество', 'group a_доля', 'group a_конверсия', 'group_a_конверсия_n_n-1']

# Таблица для группы B
event_B_pivot = event_group_pivot('B')
event_group_ratio(event_B_pivot)
event_B_pivot.columns = ['event_name', 'group b_количество', 'group b_доля', 'group b_конверсия', 'group_b_конверсия_n_n-1']

# Объединение таблиц
event_group_pivot = event_A_pivot.merge(event_B_pivot, on='event_name')

# Вывод таблицы
event_group_pivot

,event_name,group a_количество,group a_доля,group a_конверсия,group_a_конверсия_n_n-1,group b_количество,group b_доля,group b_конверсия,group_b_конверсия_n_n-1
0,login,2082,0.75,1.00,1.00,705,0.25,1.00,1.00
1,product_page,1360,0.49,0.65,0.65,397,0.14,0.56,0.56
2,product_cart,631,0.23,0.30,0.46,195,0.07,0.28,0.49
3,purchase,652,0.23,0.31,1.03,198,0.07,0.28,1.02


Поменял последовательность в `event_name`, потомучто наша целевая метрика конверисия в покупку и так как количество событий по метрике больше чем событий в `product_cart`, и существуют разные пользовательские сценарии, когда покупка может быть совершена без предварительного добавления в корзину.

In [63]:

# Создание графика воронки
fig = go.Figure()

fig.add_trace(go.Funnel(
    name='Group A',
    y=event_group_pivot['event_name'],
    x=event_group_pivot['group a_количество'],
    textposition='inside',
    textinfo='value+percent initial+percent previous'))

fig.add_trace(go.Funnel(
    name='Group B',
    y=event_group_pivot['event_name'],
    x=event_group_pivot['group b_количество'],
    textposition='inside',
    textinfo='value+percent initial+percent previous'))

fig.update_layout(
    title='Воронка событий по тестовым группам',
    funnelmode='stack', width=1200, height=600,
    yaxis_title = 'Название события',
    showlegend=True)

fig.show()

### Вывод:
При построении воронки событий мы обнаружили необычные результаты, такие как некоторые пользователи, совершающие покупку, не проходят  этап добавления товара в корзину. Это говорит о наличии разных пользовательских сценариев, когда покупка может быть совершена непосредственно, минуя шаг добавления в корзину. Это может быть вызвано особенностями продукта или пользовательским поведением

Кроме того, общий показатель конверсии в покупку для обеих групп очень низкий и составляет около 30%. Это означает, что на каждом этапе воронки мы теряем около половины пользователей. У группы B (новая платежная воронка) показатели ниже на 9% при переходе от логина к продуктовой странице и на 2% на всех остальных этапах (продуктовая карта, покупка).

## Вывод:
Проведенный исследовательский анализ данных позволяет сделать несколько ключевых наблюдений. 

По результатам анализа распределения событий между пользователями группы А и группы В можно сделать вывод о статистически значимых различиях в наблюдаемый период времени.

Распределение событий между выборками в период наблюдений с 7 декабря по 30 декабря имеет свои особенности. События группы В имеют нормальное распределение, а события группы А кратно возрастают с 15 декабря и достигают пика 21-22 декабря, а затем снижаются к 25 декабря. Распределение событий между выборками с 07.12.2020 по 14.12.2020 примерно одинаковое по количеству.
Доля пользователей из EU больше всех остальных вместе взятых. Пользователи из двух регионов EU и N.America (самые большие в количественном выражении) завершают свою активность по всем событиям 25 декабря,
а после 25 декабря активность остается только у пользователей из регионов APAC и CIS, но их доля непропорциональна мала и покупки приходятся на самую низкую цену 4.99. Группа В имеет низкую долю покупок в категории 499.99, что может указывать на особенности предпочтений этой группы.
По девайсам наибольшее количество пользователей использует Android, PC и iPhone примерно в равных долях, а Mac следует за ними.
При построении воронки событий были обнаружены некоторые проблемы , такие как число пользователей, заходящих в корзину, меньше числа пользователей, совершающих покупку. Общий показатель конверсии в покупку для обеих групп очень низкий и составляет около 30%. Это может быть связано с техническими проблемами на сайте, которые требуют дополнительного анализа и исправления. Кроме того, общий показатель конверсии в покупку для обеих групп низкий, что подразумевает потенциал для улучшения.

- Отдельно хотелось бы отметить что для успешного проведения A\B теста нужно учесть факторы которые могут повлиять на его результаты.
Наши данные находятся в периоде от 7 декабря по 30 декабря, хотя по условиям ТЗ тест мы должны были остановить 4 января 2021 года.
В нашем случае тест проводился в предрождественский сезон, который является особенно активным временем для жителей ЕС, в то время как жители других регионов готовятся к Новому году. Хотя период тестирования приходится на другие календарные даты для жителей Азии, он все равно оказывает относительное влияние. 
- Доля участников нашего теста среди новых пользователей из региона EU: 5.61% хотя по ТЗ должно быть 15%.
- В тесте присутствуют участники из других регионов кроме EU 

При проведении A/B теста следует учесть следующее:
1. Убедиться в выборке репрезентативной группы пользователей для каждой из групп А и В, необходимо определить размер выборки и уровень значимости, чтобы получить достоверные результаты тестирования.
2. Учитывать различия между группами А и В, которые могут повлиять на результаты тестирования, например, сезонность или изменения в поведении пользователей.
3. Разработать и реализовать одинаковые условия и сценарии для обеих групп, за исключением тестируемого изменения.
4. Проанализировать статистическую значимость различий между группами и сделать выводы на основе полученных результатов.

## A\B тестирование и результаты

## Проверка статистической разницы долей z-критерием.


Определим гипотезы:

In [64]:
hypothesis_H0 = "Доли уникальных посетителей, побывавших на этапе воронки, равны между собой."
hypothesis_H1 = "Между долями уникальных посетителей, побывавших на этапе воронки, есть значимая разница."

In [65]:
# Определим события для проверки
events = ['login', 'product_page', 'product_cart', 'purchase']

# Выполним z-критерий для каждого события
for event in events:
    # Создадим таблицы воронок для группы A и группы B
    funnel_A = ab_test_final[(ab_test_final['group'] == 'A') & (ab_test_final['event_name'] == event)]
    funnel_B = ab_test_final[(ab_test_final['group'] == 'B') & (ab_test_final['event_name'] == event)]
    
    # Извлечем количество успешных событий для каждой группы и события
    successes_A = funnel_A['user_id'].nunique()
    successes_B = funnel_B['user_id'].nunique()
    
    # Выполним z-критерий
    count = [successes_A, successes_B]
    nobs = [ab_test_final[(ab_test_final['event_name'] == 'login') & (ab_test_final['group'] == 'A')]['user_id'].nunique(),
            ab_test_final[(ab_test_final['event_name'] == 'login') & (ab_test_final['group'] == 'B')]['user_id'].nunique()]
    
    z_stat, p_value = proportions_ztest(count, nobs)
    
    # Выведем результаты
    print(f"Событие: {event}")
    if p_value < 0.05:
        print("Гипотеза H1: Между долями уникальных посетителей, побывавших на этапе воронки, есть значимая разница.")
    else:
        print("Гипотеза H0: Доли уникальных посетителей, побывавших на этапе воронки, равны между собой.")
    print(f"Z-статистика: {z_stat}")
    print(f"P-значение: {p_value}")
    print("-------------------------------------")

Событие: login
Гипотеза H0: Доли уникальных посетителей, побывавших на этапе воронки, равны между собой.
Z-статистика: nan
P-значение: nan
-------------------------------------
Событие: product_page
Гипотеза H1: Между долями уникальных посетителей, побывавших на этапе воронки, есть значимая разница.
Z-статистика: 4.283626682686839
P-значение: 1.8387132766752543e-05
-------------------------------------
Событие: product_cart
Гипотеза H0: Доли уникальных посетителей, побывавших на этапе воронки, равны между собой.
Z-статистика: 1.3306487024844451
P-значение: 0.18330462936588865
-------------------------------------
Событие: purchase
Гипотеза H0: Доли уникальных посетителей, побывавших на этапе воронки, равны между собой.
Z-статистика: 1.610487921566414
P-значение: 0.10729138019857214
-------------------------------------


## Вывод:
Доли уникальных посетителей , побывавших на этапе воронки, одинаковы для тестируемых групп.

## Общий вывод:

В ходе анализа данных о маркетинговых событиях, новых пользователях и A/B-тестировании были выявлены следующие факты. Некоторые пользователи одновременно участвовали в A/B-тестировании, маркетинговом событии "Christmas&New Year Promo" и другом A/B-тесте. Эти пользователи были исключены из анализа, так как одновременные события могли искажать результаты. Таким образом, данные были очищены от пересекающихся событий, что составило примерно 40% от общего объема данных.

Кроме того, обнаружено, что группы A и B включают в себя разные по долям количесво пользователей. Группа A состоит примерно из 75% всех пользователей (2 082 человек), в то время как группа B - около 25% (706 человек). Идеальное соотношение между группами должно быть 50% к 50%. Анализ распределения факторов внутри групп показал, что можно достичь более равномерного распределения, если рассматривать период до 14 декабря 2020 года. Однако это приведет к значительному сокращению выборки до примерно 60% в группе A и 40% в группе B. Несмотря на неидеальное распределение пользователей между группами, нет других факторов, которые можно было бы исключить.

Также обнаружено, что доля пользователей из региона EU превышает ожидания задания, вообще является доминирующей, количество пользователей этой группы превосходит всех других вместе взятых. Это указывает на особенности предпочтений пользователей и может потребовать дополнительного исследования.

Анализ воронки событий выявил проблемы в технической реализации логики сайта, например, число пользователей, переходящих в корзину, меньше числа пользователей, совершающих покупку. Общий показатель конверсии в покупку для обеих групп низкий и составляет около 30%. Это может быть связано с техническими проблемами, требующими дополнительного анализа и исправления. Также выявлено, что группа B имеет низкую долю покупок в категории 499.99, что может указывать на особенности предпочтений этой группы.

Проверка статистической разницы долей с помощью z-критерия показала, что доли уникальных посетителей на этапе воронки равны для обеих тестируемых групп, кроме события `product_page`, где между долями уникальных посетителей, побывавших на этом этапе воронки, есть значимая разница. Это может свидетельствовать о том, что изменения, внесенные в тестируемую группу, имеют влияние на привлечение посетителей на этот конкретный этап воронки. Для более подробного анализа  рекомендуется провести дополнительные исследования и рассмотреть другие факторы, которые могут влиять на поведение посетителей на этом этапе.

Учитывая, что тест проводился в предрождественский сезон, активный для жителей ЕС, в то время как жители других регионов готовятся к Новому году, и с учетом того, что не все условия ТЗ были выполнены, запустить новый тест с дополнительным анализом и исправлением выявленных проблем.


